# Predicting Student Depression: A Big Data Analytics Approach with Apache Spark

- **Author:** David Araba
- **Student ID:** 48093143
- **Course:** INFS3208 - Cloud Computing
- **Date:** October 2025

## 1. Introduction & Project Goals

### 1.1. Objective
The primary objective of this project is to develop and evaluate a suite of machine learning models using Apache Spark to predict the likelihood of depression among students. By leveraging a comprehensive dataset that includes demographic, academic, and lifestyle factors, I aim to identify key indicators associated with mental health challenges in an academic environment.

### 1.2. Significance
Student mental health is a growing concern globally. The pressures of academic life, combined with financial and social stressors, can significantly impact a student's well-being and academic performance. This project is important because it seeks to create a data-driven framework that could potentially identify at-risk students, enabling educational institutions to offer timely and targeted support. By using scalable cloud computing technologies, we can build a foundation for a system capable of handling large-scale, real-world student data, moving from reactive to proactive mental wellness strategies. This aligns with the need for modern solutions that traditional computing can struggle to scale effectively.

### 1.3. Technical Stack
This project will be implemented using the following technologies:
* **Language:** Python 3.x
* **Core Engine:** Apache Spark (via PySpark)
* **Libraries:**
    * **Spark MLlib:** For building scalable machine learning pipelines.
    * **Pandas:** For initial data handling and manipulation.
    * **Matplotlib & Seaborn:** For data visualisation and result interpretation.

## 2. Project Architecture & Workflow


### 2.1. Workflow Description
This project follows a standard big data analytics workflow, as depicted in the diagram below. The process begins with the ingestion of four separate but related data files into the Spark environment. These datasets are then joined and pre-processed to create a unified, analysis-ready master dataset. Subsequently, this dataset is used to train and evaluate four distinct machine learning functionalities as required by the project specification: classification, regression, clustering, and association rule mining. The final insights and model performance metrics are then visualised to provide clear, interpretable results.

### 2.2. Architecture Diagram
This diagram illustrates the project's workflow from data source to final analysis. It explicitly shows the use of multiple data sources and the application of various Spark MLlib functionalities, fulfilling the key project requirements.


### 2.3. Architecture Explanation
The above workflow diagram demonstrates a comprehensive big data analytics pipeline designed for scalable student mental health analysis. The process begins with four distinct data sources being ingested simultaneously into the Spark environment, leveraging Spark's distributed processing capabilities to handle large-scale datasets efficiently. The unified master dataset is then processed through feature engineering pipelines before being split for model training and evaluation. The four ML functionalities operate in parallel, each addressing different aspects of student mental health prediction and analysis. This architecture showcases the power of cloud computing technologies in handling complex, multi-dimensional data analysis tasks that would be challenging with traditional single-machine approaches.


## 3. Environment Setup
This section prepares the notebook's environment. The first part imports all necessary libraries for the project, including `pyspark` for distributed data processing, `pyspark.ml` for machine learning, and `matplotlib` for visualisation. The second part initialises the `SparkSession`, which is the essential entry point to all of Spark's functionalities.

In [1]:
# --- 3.1. Import All Necessary Libraries ---

# Standard Python libraries for data handling and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# IMPORTANT: Add this magic command to render plots inline
%matplotlib inline

# Spark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, avg, mean, array
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, ClusteringEvaluator
from pyspark.ml.fpm import FPGrowth

print("Libraries imported successfully and matplotlib is set to inline.")

# --- 3.2. Initialise Spark Session with Full GCS Configuration ---

spark = SparkSession.builder \
    .appName("StudentMentalHealthPrediction") \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.20") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.type", "APPLICATION_DEFAULT_CREDENTIALS") \
    .getOrCreate()

print(f"Spark session created successfully. Version: {spark.version}")

Libraries imported successfully and matplotlib is set to inline.


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/19 16:15:11 WARN Utils: Your hostname, Davids-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.55 instead (on interface en0)
25/10/19 16:15:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/david/Projects/uni/infs3208/SMH/smh/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/david/.ivy2.5.2/cache
The jars for the packages stored in: /Users/david/.ivy2.5.2/jars
com.google.cloud.bigdataoss#gcs-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19b1a578-0836-41d0-8f18-cf6c0110f0fc;1.0
	confs: [default]
	found com.google.cloud.bigdataoss#gcs-connector;hadoop3-2.2.20 in central
	found com.google.api-client#google-api-client-jackson2;2.0.1 in central
	found com.google.api-client#go

Spark session created successfully. Version: 4.0.1


## Part 1: Data Loading and Exploratory Data Analysis (EDA)

This section focuses on the initial steps of our data science workflow as outlined in the project structure. We will load our datasets, merge them into a unified DataFrame, conduct an initial inspection to understand its structure and quality, and then perform exploratory visualizations to uncover key patterns.

### 1.1 Ingest and Merge Datasets

First, we load the four separate CSV files (`student_info.csv`, `academic_data.csv`, `lifestyle_data.csv`, and `mental_health.csv`) into individual Spark DataFrames. We then perform a series of inner joins on the `id` column to create a single, unified master dataset for our analysis.

In [ ]:

# --- 1.1 Ingest and Merge Datasets ---

# --- Configuration Switch ---
# Set this to True to read from Google Cloud Storage
# Set this to False to read from the local '../data/' directory
USE_GCS = True

# --- Define File Paths based on the Switch ---
if USE_GCS:
    bucket_name = "david-araba-infs3208-data"
    base_path = f'gs://{bucket_name}/'
    print(f"✅ Reading data from Google Cloud Storage bucket: {bucket_name}")
else:
    base_path = "../data"
    print("✅ Reading data from local '../data/' directory.")

# Define the full paths for each file
path_info = base_path + 'student_info.csv'
path_academic = base_path + 'academic_data.csv'
path_lifestyle = base_path + 'lifestyle_data.csv'
path_mental = base_path + 'mental_health.csv'

# --- Load Data from the Selected Source ---
try:
    sdf_info = spark.read.csv(path_info, header=True, inferSchema=True)
    sdf_academic = spark.read.csv(path_academic, header=True, inferSchema=True)
    sdf_lifestyle = spark.read.csv(path_lifestyle, header=True, inferSchema=True)
    sdf_mental = spark.read.csv(path_mental, header=True, inferSchema=True)

    # Perform a series of inner joins on the 'id' column
    master_df = sdf_info.join(sdf_academic, "id", "inner") \
                        .join(sdf_lifestyle, "id", "inner") \
                        .join(sdf_mental, "id", "inner")

    # Show the first 5 rows and verify the total count to confirm the merge
    print("\nSuccessfully merged all data sources. Displaying a sample:")
    master_df.show(5)
    print(f"The master DataFrame contains {master_df.count()} rows.")

except Exception as e:
    print("\n⚠️ An error occurred while loading the data.")
    if USE_GCS:
        print("Please check the following:")
        print("1. Your GCS bucket name is correct.")
        print("2. You are authenticated with Google Cloud (e.g., via 'gcloud auth application-default login').")
        print("3. The CSV files exist in the bucket.")
    else:
        print("Please check that the '../data/' directory and all CSV files exist.")
    print(f"\nError details: {e}")

✅ Reading data from Google Cloud Storage bucket: david-araba-infs3208-data


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/david/Projects/uni/infs3208/SMH/smh/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/david/Projects/uni/infs3208/SMH/smh/lib/python3.13/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/david/miniconda3/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt
25/10/19 16:18:26 WARN FileSystem: Failed to initialize filesystem gs://david-araba-infs3208-data/student_info.csv: java.io.IOException: Error getting access token from metadata server at: http://169.254.169.254/computeMetadata/v1/instance/service-accounts/default/token
25/10/19 16:18:26 WARN FileStreamSink: Assume no metadata dire

KeyboardInterrupt: 

25/10/19 16:18:46 WARN FileSystem: Failed to initialize filesystem gs://david-araba-infs3208-data/student_info.csv: java.io.IOException: Error getting access token from metadata server at: http://169.254.169.254/computeMetadata/v1/instance/service-accounts/default/token


### 1.2 Initial Data Inspection

Now that the data is merged, we perform an initial inspection to understand its structure and identify potential data quality issues. We use `.printSchema()` to review column names and data types and `.describe().show()` to get a statistical summary of the numerical columns. This step is crucial for spotting inconsistencies, such as numerical data being incorrectly read as strings.

In [ ]:
# Print the schema to check column names and data types
print("Schema of the master DataFrame:")
master_df.printSchema()

# Get a statistical summary of the numerical features
print("\nStatistical summary of numerical features:")
master_df.describe().show()

Schema of the master DataFrame:


NameError: name 'master_df' is not defined

The schema and summary statistics immediately highlight a data quality issue. While most columns are correctly typed, the `.describe()` output shows a `'?'` value in the `max` row for `Financial Stress`, confirming it was incorrectly read as a `string`. This prevents proper statistical analysis and must be corrected.

### 1.3 Data Type Correction and Quality Check

The initial inspection confirmed that columns intended to be numerical, such as `Financial Stress`, were incorrectly inferred as a `string` type due to `'?'` placeholder values. To rectify this, we will explicitly cast all numerical columns to a `double` type. Using `try_cast` is essential as it will gracefully convert these placeholders into `NULL` values.

After this correction, we'll perform a systematic check for null values to see the full extent of the missing data that needs to be handled in the pre-processing stage.

In [ ]:
# --- Data Type Correction and Column Definitions ---
categorical_cols = [
    'Gender', 'City', 'Profession', 'Degree', 'Sleep Duration', 
    'Dietary Habits', 'Have you ever had suicidal thoughts ?', 
    'Family History of Mental Illness'
]

numerical_cols = [
    'Age', 'CGPA', 'Academic Pressure', 'Work Pressure',
    'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress'
]

# Create a copy of the master DataFrame to work with
df_corrected = master_df

# Explicitly cast numerical columns to a Double type, converting non-numeric values to NULL
for column in numerical_cols:
    df_corrected = df_corrected.withColumn(column, 
        expr(f"try_cast(`{column}` as double)")
    )

print("✅ Data types corrected successfully.")
print("\nVerifying the new schema:")
df_corrected.printSchema()

# --- Data Quality Check ---
# Now, check for nulls on the CORRECTED DataFrame
null_counts = [(c, df_corrected.where(col(c).isNull()).count()) for c in df_corrected.columns]

# Filter and display only columns that have at least one null value
null_df = pd.DataFrame(null_counts, columns=['Column', 'Null_Count'])
print("\nChecking for missing values after type correction:")
print(null_df[null_df['Null_Count'] > 0])

if null_df['Null_Count'].sum() == 0:
    print("\n✅ Great! No missing values found in the dataset.")
else:
    print("\n⚠️ Missing values were found. These will be handled in the pre-processing step.")

The output confirms the data type correction was successful, with the schema now showing `Financial Stress` as a `double`. The subsequent null check demonstrates the effect of `try_cast`: the 3 non-numeric placeholder values have been correctly identified and converted into `NULL`. This provides a precise count of the missing data that must be addressed in the pre-processing step.

### 1.4 Exploratory Visualisation

With our data loaded and inspected, we now create visualizations to better understand the distributions of key features. This includes analyzing the balance of our target variable ('Depression') and exploring the distributions of important demographic and academic features. For ease of plotting, we convert the cleansed Spark DataFrame (`df_corrected`) to a Pandas DataFrame.

In [ ]:
# --- Target Variable Distribution ---

# Aggregate the data in Spark to count occurrences of each class in the 'Depression' column
depression_counts = df_corrected.groupBy('Depression').count().toPandas()

# Map the numerical labels to meaningful names for the plot
depression_counts['Depression'] = depression_counts['Depression'].map({0: 'Not Depressed', 1: 'Depressed'})

# Create the plot using Seaborn with specified colors
plt.figure(figsize=(8, 6))
# Define a custom color palette
custom_palette = {'Depressed': 'red', 'Not Depressed': 'blue'}
sns.barplot(x='Depression', y='count', data=depression_counts, palette=custom_palette, order=['Depressed', 'Not Depressed'])
plt.title('Distribution of Student Depression Status', fontsize=16)
plt.xlabel('Status', fontsize=12)
plt.ylabel('Number of Students', fontsize=12)
plt.show()

The bar chart of our target variable reveals a moderate class imbalance. There are more students classified as 'Depressed' than 'Not Depressed' in this dataset. This is a critical observation that will influence how we evaluate our classification models later, as simple accuracy could be a misleading metric.

In [ ]:
# Convert the corrected Spark DataFrame to a Pandas DataFrame for easier plotting
pandas_df = df_corrected.toPandas()

# --- Create a figure with multiple subplots for feature distributions ---
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle('Distribution of Key Demographic and Academic Features', fontsize=18)

# 1. Gender Distribution (Code updated to resolve FutureWarning)
sns.countplot(ax=axes[0], x='Gender', data=pandas_df, hue='Gender', palette='viridis', legend=False)
axes[0].set_title('Gender Distribution')
axes[0].set_xlabel('Gender')
axes[0].set_ylabel('Count')

# 2. Age Distribution
sns.histplot(ax=axes[1], x='Age', data=pandas_df, kde=True, color='skyblue', bins=30)
axes[1].set_title('Age Distribution of Students')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')

# 3. CGPA Distribution
sns.histplot(ax=axes[2], x='CGPA', data=pandas_df, kde=True, color='salmon')
axes[2].set_title('Distribution of CGPA')
axes[2].set_xlabel('CGPA')
axes[2].set_ylabel('Frequency')

# Adjust layout and display the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make space for suptitle
plt.show()

These visualizations provide further insights into the dataset's composition:
* **Gender:** The dataset contains a higher number of male participants than female participants.
* **Age:** The age distribution is concentrated between 18 and 30 years, as expected for a student population, with several distinct peaks.
* **CGPA:** The CGPA distribution is highly unusual. It is left-skewed with sharp, distinct peaks at integer and half-integer values (e.g., 7.0, 7.5, 8.0). This strongly suggests that CGPA was either self-reported with rounding or originates from a discrete grading system, rather than being a continuous measure.

## Part 2: Feature Engineering & Pre-processing

This section prepares our clean, explored data for the machine learning models. The primary goal is to transform the dataset into a format suitable for Spark's MLlib library. This involves three key steps:

1. **Data Cleaning:** We will implement the strategy decided upon in Part 1 by handling the `NULL` values discovered in the `Financial Stress`.
2.  **Feature Transformation:** Machine learning models require numerical inputs, so we must convert our categorical text columns (like `Gender`, `City`, etc.) into a numerical format. We will use a standard two-step process with **`StringIndexer`** and **`OneHotEncoder`**.
3.  **Feature Assembling:** Finally, we will use **`VectorAssembler`** to combine all our individual feature columns into a single vector column. This is a mandatory step, as all Spark MLlib models expect this specific format for their input features.

### 2.1 Data Cleaning: Imputing Missing Values

As identified in the EDA phase, our type correction process created three `NULL` values in the `Financial Stress` column. Leaving these as `NULL` would cause errors during model training. A common and effective strategy for handling a small number of missing numerical values is imputation. Here, we replace the `NULL` values with the statistical **mean** of the entire column, preserving the overall distribution of the data without discarding valuable rows.

In [ ]:
# --- Handle Missing Values Created from try_cast ---

# Check for null values created by try_cast (from '?' values)
null_counts = [(c, df_corrected.where(col(c).isNull()).count()) for c in numerical_cols]
null_df = pd.DataFrame(null_counts, columns=['Column', 'Null_Count'])
print("Checking for null values created from try_cast:")
print(null_df[null_df['Null_Count'] > 0])

# Fill null values with column means
for column in numerical_cols:
    # Calculate mean for each column
    mean_val = df_corrected.select(mean(col(column))).collect()[0][0]
    if mean_val is not None:  # Only fill if mean exists
        df_corrected = df_corrected.fillna(mean_val, subset=[column])
        print(f"Filled null values in {column} with mean: {mean_val:.2f}")

print("✅ Data cleaning complete. All '?' values converted to column means.")

### 2.2 Feature Transformation: Encoding Categorical Variables

Machine learning algorithms require numerical input. Therefore, we must convert our text-based categorical columns (e.g., 'Gender', 'City') into a numerical format. We use a standard two-step process for each column:

1.  **`StringIndexer`**: This transformer assigns a unique numerical index to each unique category in a column (e.g., 'Male' -> 0.0, 'Female' -> 1.0).
2.  **`OneHotEncoder`**: This transformer takes the numerical index and converts it into a sparse binary vector. This prevents the model from incorrectly assuming an ordinal relationship between categories (e.g., that 'City B' is somehow "greater than" 'City A').

We create these two stages for every categorical column and add them to our pipeline.

In [ ]:
# --- Create a list of transformation stages ---
stages = []

# Iterate over each categorical column to create StringIndexer and OneHotEncoder stages
for column in categorical_cols:
    string_indexer = StringIndexer(inputCol=column, outputCol=column + "_Index")
    one_hot_encoder = OneHotEncoder(inputCols=[string_indexer.getOutputCol()], outputCols=[column + "_Vec"])
    stages += [string_indexer, one_hot_encoder]

print("StringIndexer and OneHotEncoder stages created.")

### 2.3 Feature Assembling

All Spark MLlib models expect the input features to be consolidated into a single vector column. We use `VectorAssembler` to achieve this. This final transformation stage gathers all our processed feature columns—both the original numerical columns and the new one-hot encoded vectors—and combines them into a single column named `features`.

In [ ]:
# Create a list of all feature columns (numerical + the new vector columns from OHE)
feature_cols = numerical_cols + [c + "_Vec" for c in categorical_cols]

# Create the VectorAssembler stage
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Add the assembler to our list of stages
stages += [vector_assembler]

print("VectorAssembler stage created.")

### 2.4 Executing the Pre-processing Pipeline

With all our stages defined (imputation handled separately, followed by indexing, encoding, and assembling), we combine them into a single `Pipeline`. A `Pipeline` is a powerful Spark MLlib tool that chains multiple transformers and estimators together to create a unified workflow.

We then `.fit()` the pipeline on our data to learn the necessary transformations (like the string-to-index mappings) and `.transform()` the data to apply them. The output below shows the final structure of our dataset, now ready for model training, with the `features` vector successfully created.

In [ ]:
# Create the full pre-processing pipeline with all our defined stages
preprocessing_pipeline = Pipeline(stages=stages)

# Fit the pipeline to our corrected data
pipeline_model = preprocessing_pipeline.fit(df_corrected)

# Transform the data to apply all the steps
transformed_df = pipeline_model.transform(df_corrected)

# Display the results to confirm the 'features' vector was created
print("Data transformation complete. Displaying the transformed DataFrame:")
transformed_df.select('id', 'Depression', 'features').show(5, truncate=False)

## Part 3: Machine Learning Modelling

This is the core implementation section of the project where we build, train, and evaluate our machine learning models. With our data now fully pre-processed, we will address a key assignment requirement by implementing **four distinct analytical tasks** using Spark MLlib.

The goal is to analyze the student dataset from multiple perspectives to uncover different types of insights. Our four chosen functionalities, as outlined in the project proposal, are:

* **1. Classification:** Predicting the likelihood of a student having depression.
* **2. Regression:** Predicting a student's academic performance (CGPA).
* **3. Clustering:** Identifying distinct groups or profiles of students based on their data.
* **4. Association Rule Mining:** Discovering hidden patterns or relationships between different factors.

Our first crucial step will be to split the data into training and testing sets. This ensures we can evaluate our supervised models on unseen data, providing an unbiased assessment of their predictive power.

### 3.1 Data Splitting

Before we can train any supervised machine learning models, we must prepare our dataset. This involves two key actions:

1.  **Renaming the Target Column:** Spark MLlib's classification and regression algorithms expect the target variable (the value we want to predict) to be in a column named `label`. We rename our `Depression` column to `label` to adhere to this convention.
2.  **Splitting the Data:** We split our dataset into a **training set (80%)** and a **testing set (20%)**. The model will only ever learn from the training data. The testing data is held back as "unseen" data to provide an unbiased evaluation of how well the model generalizes to new, real-world examples. We use a `seed` to ensure this random split is the same every time we run the notebook, making our results reproducible.

In [ ]:
# Rename the target column 'Depression' to 'label'
final_df = transformed_df.withColumnRenamed("Depression", "label")

# Split the data into training and testing sets (80% training, 20% testing)
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)

# Print the number of records in each set to verify the split
print(f"Number of training records: {train_data.count()}")
print(f"Number of testing records: {test_data.count()}")


### 3.2 Functionality 1: Classification - Predicting Depression

Our first analytical task is classification. We will train a **Logistic Regression** model to predict our binary target variable: whether a student is likely to have depression (`1`) or not (`0`). Logistic Regression is a robust and highly interpretable algorithm, making it an excellent baseline model for this kind of problem.

The process involves two main steps:

1.  **Training:** We initialize the `LogisticRegression` model, specifying the input `features` and `label` columns. We then train the model by calling the `.fit()` method on our `train_data`.
2.  **Prediction:** Once the model is trained, we use the `.transform()` method on the unseen `test_data` to generate predictions.

The output table shows the result. For each row in the test set, we can see the true `label`, the model's final `prediction` (0 or 1), and the `probability` vector, which indicates the model's confidence in each class.

In [ ]:
# --- 1. Initialize and Train the Logistic Regression Model ---
print("Training Logistic Regression model...")
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)
print("Model training complete.")

# --- 2. Make Predictions and Format the Output ---
print("\nMaking predictions on the test data...")
predictions = lr_model.transform(test_data)

# Create a UDF to extract the probability of the positive class (class 1)
extract_prob = udf(lambda v: float(v[1]), FloatType())

# Apply the UDF to create a 'Confidence' column and round it
predictions_formatted = predictions.withColumn("Confidence", spark_round(extract_prob("probability"), 4)) \
                                   .select("label", "prediction", "Confidence")

# Show the new, user-friendly table
print("\nUser-Friendly Prediction Results:")
predictions_formatted.show(10)

### Understanding the Model's Predictions

The table below analyzes the first 10 predictions from your test set. The key is the **Confidence** column, which represents the model's calculated probability that a student *is depressed* (the probability of `label = 1`). The model's final **Prediction** is `1.0` if this confidence is above 50% (0.5) and `0.0` if it's below.

| Row | Label (True Answer) | Confidence P(Depressed) | Prediction (Model's Guess) | Result |
| :-- | :------------------ | :---------------------- | :------------------------- | :----- |
| 1   | 0 (Not Depressed)   | 2.77%                   | 0.0                        | ✅ Correct |
| 2   | 0 (Not Depressed)   | 2.88%                   | 0.0                        | ✅ Correct |
| 3   | 1 (Depressed)       | **85.31%** | 1.0                        | ✅ Correct |
| 4   | 0 (Not Depressed)   | 23.83%                  | 0.0                        | ✅ Correct |
| 5   | 1 (Depressed)       | 31.51%                  | 0.0                        | ❌ Incorrect |
| 6   | 0 (Not Depressed)   | 4.72%                   | 0.0                        | ✅ Correct |
| 7   | 1 (Depressed)       | **80.86%** | 1.0                        | ✅ Correct |
| 8   | 0 (Not Depressed)   | 16.32%                  | 0.0                        | ✅ Correct |
| 9   | 0 (Not Depressed)   | 9.35%                   | 0.0                        | ✅ Correct |
| 10  | 0 (Not Depressed)   | **66.46%** | 1.0                        | ❌ Incorrect |

This row-by-row analysis provides clear intuition for the model's behavior. For example, in **Row 3**, the model was **85.31%** confident the student was depressed, so it correctly predicted `1.0`, matching the true label. Conversely, in **Row 10**, the model was 66.46% confident and predicted `1.0`, but the true answer was `0`, making it confidently wrong. The next step is to aggregate these results to see the overall performance across the entire test set.

### 3.2.1 Evaluating the Classification Model

Now that we have predictions, we need to evaluate how well our Logistic Regression model performed. We will use several standard classification metrics to get a complete picture of its strengths and weaknesses. A single metric like accuracy can be misleading, especially with imbalanced datasets.

In [ ]:
# --- 1. Evaluate using AUC-ROC ---
evaluator_roc = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
auc = evaluator_roc.evaluate(predictions)
print(f"Area Under ROC Curve (AUC): {auc:.4f}")

# --- 2. Calculate TP, TN, FP, FN from the Predictions DataFrame ---
# These values are the building blocks for all other metrics
tp = predictions.filter((col('label') == 1) & (col('prediction') == 1)).count()
tn = predictions.filter((col('label') == 0) & (col('prediction') == 0)).count()
fp = predictions.filter((col('label') == 0) & (col('prediction') == 1)).count()
fn = predictions.filter((col('label') == 1) & (col('prediction') == 0)).count()

# --- 3. Calculate and Print Key Performance Metrics ---
accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nAccuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1_score:.4f}")

# --- 4. Display a Formatted Confusion Matrix ---
# Create a more descriptive confusion matrix using Pandas for better visualization
confusion_matrix_pd = pd.DataFrame(
    data=[[tp, fn], [fp, tn]],
    columns=['Predicted: Depressed', 'Predicted: Not Depressed'],
    index=['Actual: Depressed', 'Actual: Not Depressed']
)

print("\n--- Formatted Confusion Matrix ---")
display(confusion_matrix_pd)

### Interpreting the Model's Performance

These results show our model is performing quite well as a baseline for identifying students with depression. 📈

#### Area Under ROC Curve (AUC): 0.9230 🎯
This is an **excellent score**. The AUC measures how well the model can distinguish between a depressed and a non-depressed student. A score of 0.5 is random guessing, while 1.0 is perfect. Your score of **0.923** indicates the model has a very strong discriminative ability.

---
#### Confusion Matrix Breakdown 📊
The confusion matrix gives us a detailed breakdown of the model's successes and errors:

* **True Positives (TP): 2,798**
    * Students who **were depressed** and were **correctly** identified by the model. ✅
* **True Negatives (TN): 1,889**
    * Students who were **not depressed** and were **correctly** identified. ✅
* **False Positives (FP): 486**
    * Students who were **not depressed** but were **incorrectly** flagged as depressed (a "false alarm"). ❌
* **False Negatives (FN): 373**
    * Students who **were depressed** but were **missed** by the model. This is the most critical error in a health context. ❌

---
#### Key Performance Metrics
These percentages translate the raw numbers from the matrix into performance scores:

* **Accuracy: 84.51%**
    * Overall, **84.51%** of the model's predictions were correct. It's a solid score but doesn't tell the full story.
* **Precision: 85.20%**
    * When the model predicted a student was depressed, it was correct **85.20%** of the time. This tells you how reliable a "positive" prediction is.
* **Recall: 88.24%**
    * Of all the students who **truly were depressed**, the model successfully identified **88.24%** of them. This is a crucial metric for this project, as a high recall means we are minimizing the number of at-risk students we miss.
* **F1 Score: 0.8669**
    * This is the balanced average of Precision and Recall. A score of **0.867** indicates a strong and well-balanced model, effective at handling both false positives and false negatives.

The high recall is especially promising for a mental health application, though the 373 false negatives highlight an area for future improvement.

### 3.3 Functionality 2: Regression - Predicting Academic Performance (CGPA)

Our second analytical task is **regression**, where the goal is to predict a continuous numerical value instead of a categorical class. For this task, we will train a **Linear Regression** model to predict a student's **CGPA** based on all other features.

This is a valuable exercise for two reasons:
1.  **Insight Generation:** It can help us understand which lifestyle, demographic, or mental health factors are most strongly correlated with academic performance.
2.  **Technical Requirement:** It fulfills another of the four required functionalities for this project.

The process will be similar to our classification task: we'll train the model on the training data, make predictions on the test data, and then evaluate the model's performance using regression-specific metrics like Root Mean Squared Error (RMSE) and R-squared (R²).

### 3.3.1 Preparing Data and Training the Regression Model

Before training our regression model, we must perform a crucial data preparation step to prevent **data leakage**. Our goal is to predict `CGPA`, so we must remove it from the input features. Including the target variable in the feature set would make the model's job trivial and produce misleadingly perfect results.

The workflow is as follows:
1.  **Create a New Feature Set:** We define a new list of numerical columns that excludes `CGPA`.
2.  **Re-Assemble Features:** We use a new `VectorAssembler` to create a `regression_features` vector using this updated list.
3.  **Prepare Final DataFrame:** We create a new DataFrame specifically for this task, selecting our new features and renaming `CGPA` to `label` for compatibility with the Spark MLlib model.
4.  **Train and Predict:** We split this new DataFrame into training and testing sets and then fit a `LinearRegression` model to predict the `label` (CGPA).

In [ ]:
# --- 1. Prepare Data for Regression ---
print("Preparing data for regression task...")
# We must exclude CGPA from our features since it's our new target variable.
regression_numerical_cols = [
    'Age', 'Academic Pressure', 'Work Pressure',
    'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress'
]
regression_feature_cols = regression_numerical_cols + [c + "_Vec" for c in categorical_cols]
regression_vector_assembler = VectorAssembler(inputCols=regression_feature_cols, outputCol="regression_features")
regression_features_df = regression_vector_assembler.transform(transformed_df)
regression_df = regression_features_df.select(col("regression_features").alias("features"), col("CGPA").alias("label"))
reg_train_data, reg_test_data = regression_df.randomSplit([0.8, 0.2], seed=42)
print("Data preparation complete.")

# --- 2. Initialize and Train the Linear Regression Model ---
print("\nTraining Linear Regression model...")
lr_regressor = LinearRegression(featuresCol="features", labelCol="label")
lr_regressor_model = lr_regressor.fit(reg_train_data)
print("Model training complete.")

# --- 3. Make Predictions and Format the Output ---
print("\nMaking predictions on the test data...")
reg_predictions = lr_regressor_model.transform(reg_test_data)

# Round the prediction for readability and add an 'Error' column
reg_predictions_formatted = reg_predictions.withColumn("prediction_rounded", spark_round("prediction", 2))
reg_predictions_formatted = reg_predictions_formatted.withColumn("Error",
    spark_round(spark_abs(col("label") - col("prediction_rounded")), 2)
)

print("\n--- User-Friendly Regression Results ---")
reg_predictions_formatted.select(col("label").alias("Actual_CGPA"), col("prediction_rounded").alias("Predicted_CGPA"), "Error").show(10)

# --- 4. Evaluate the Regression Model ---
print("\n--- Model Performance Metrics ---")
reg_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label")

# Calculate RMSE
rmse = reg_evaluator.setMetricName("rmse").evaluate(reg_predictions)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Calculate R-squared
r2 = reg_evaluator.setMetricName("r2").evaluate(reg_predictions)
print(f"R-squared (R²): {r2:.4f}")

### Interpreting the Regression Results

The output from our regression model provides several key insights into both the model's behavior and the nature of our data.

#### Analyzing the Predictions 
The results table clearly shows the model's poor performance. Notice that the **Predicted_CGPA** values are all clustered in a very narrow range (around 7.5), regardless of the **Actual_CGPA**. For instance, it predicts a CGPA of **7.52** for a student who actually achieved a **5.58**, and a similar **7.43** for a student who achieved a **9.93**.

This demonstrates that the model has simply learned to guess the average CGPA for almost everyone. It reveals a crucial finding: **the lifestyle, demographic, and mental health features in our dataset are not strong predictors of academic performance**.

---
#### Evaluating the Model with Metrics 
The formal metrics numerically confirm this observation:

* **Root Mean Squared Error (RMSE): 1.4631**
    * This means that, on average, the model's prediction for a student's CGPA is off by about **1.46 points**. On a 10-point scale, this is a very significant error, confirming the model's low accuracy.
* **R-squared (R²): 0.0168**
    * This is the most definitive metric. An R² of 0.0168 means that our model can only explain about **1.68%** of the variation in student CGPA. An R² this close to zero provides strong statistical evidence that our model is performing no better than a simple baseline that just predicts the average CGPA for every student.

In conclusion, while the model runs correctly, its very high RMSE and near-zero R² score prove that it has almost no predictive power. This is a valuable finding in itself, demonstrating that the chosen factors are poor predictors for a student's CGPA using a linear model.

### 3.4 Functionality 3: Clustering - Identifying At-Risk Student Groups 

Our third analytical task is **clustering**, which is a type of **unsupervised learning**. Unlike our previous models, clustering does not use a `label` column to make predictions. Instead, its goal is to discover natural groupings or "clusters" within the data based on the similarity of the features.

For this project, we will use the **KMeans** algorithm. KMeans works by partitioning the data into a pre-defined number (`k`) of clusters, where each student belongs to the cluster with the nearest mean (a "centroid"). This is an excellent way to identify distinct student profiles, such as "High-Achieving, High-Stress" or "Socially Engaged, Low-Pressure" students.

The primary challenge with KMeans is choosing the optimal number of clusters, `k`. To do this scientifically, we will use the **Elbow Method**. This involves running the KMeans algorithm for a range of `k` values and plotting a cost function (the Within Set Sum of Squared Errors). The "elbow" of the resulting curve—the point where the rate of improvement slows down—suggests the best value for `k`.

### 3.4.1 Finding the Optimal Number of Clusters (k) with the Elbow Method

A critical parameter for the KMeans algorithm is `k`, the number of clusters. Choosing this value arbitrarily can lead to meaningless results. To make an informed decision, we will use the **Elbow Method**.

This method involves running the KMeans algorithm for a range of `k` values (e.g., from 2 to 10) and calculating the **Within Set Sum of Squared Errors (WSSSE)** for each. The WSSSE measures the total distance of all points from their respective cluster centroids; a smaller WSSSE means the clusters are denser.

We then plot these WSSSE values against `k`. The resulting curve typically looks like an arm. The point where the curve bends—the "elbow"—represents the point of diminishing returns, where adding more clusters no longer significantly reduces the WSSSE. This point is considered the optimal value for `k`.

In [ ]:
# --- Find the optimal 'k' using the Elbow Method ---
print("Calculating WSSSE for a range of k values...")

# We will test k from 2 to 10
wssse_values = []
k_values = range(2, 11)

for k in k_values:
    kmeans = KMeans(featuresCol='features', k=k, seed=42)
    model = kmeans.fit(final_df)
    # The cost is the WSSSE
    wssse = model.summary.trainingCost
    wssse_values.append(wssse)
    print(f"k={k}, WSSSE={wssse:.2f}")

# --- Plot the Elbow Method Curve ---
plt.figure(figsize=(10, 6))
plt.plot(k_values, wssse_values, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Within Set Sum of Squared Errors (WSSSE)')
plt.title('The Elbow Method for Optimal k')
plt.xticks(k_values)
plt.grid(True)
plt.show()

The Elbow Method plot clearly shows a distinct "bend" at **k=4**. After this point, the curve begins to flatten, and the reduction in WSSSE for each additional cluster becomes much less significant. This provides strong evidence that **4** is the optimal number of clusters for segmenting our student data.

### 3.4.2 Training and Evaluating the Final KMeans Model

Based on the Elbow Method analysis, we now train our final KMeans model using `k=4`. After training, we will:

1.  **Assign Clusters:** Use the trained model to assign each student in our dataset to one of the four clusters.
2.  **Evaluate Performance:** Calculate the **Silhouette Score**. This metric measures how well-separated the clusters are. A score closer to 1 is better, while a score near 0 indicates overlapping clusters.
3.  **Analyze Cluster Sizes:** Examine the number of students assigned to each cluster to ensure the groups are of a meaningful size.

In [ ]:
# --- 1. Initialize and Train the K-Means Model ---
# We select k=4 based on the Elbow Method plot above.
OPTIMAL_K = 4
print(f"Training final K-Means model with k={OPTIMAL_K}...")

kmeans = KMeans(featuresCol="features", k=OPTIMAL_K, seed=42)
kmeans_model = kmeans.fit(final_df)
print("Model training complete.")

# --- 2. Make Predictions (Assign Clusters) ---
clustered_df = kmeans_model.transform(final_df)
print("\n--- Sample of students with their assigned cluster ---")
clustered_df.select("id", "label", "prediction").show(10)

# --- 3. Evaluate the Clustering ---
evaluator = ClusteringEvaluator(predictionCol="prediction")
silhouette = evaluator.evaluate(clustered_df)
print(f"\nSilhouette Score = {silhouette:.4f}")

# --- 4. Analyze Cluster Sizes ---
print("\n--- Distribution of students across clusters ---")
clustered_df.groupBy("prediction").count().orderBy('prediction').show()

### Interpreting the Clustering Results

Our KMeans model has successfully segmented the students into four groups, but the results suggest these groups are not perfectly distinct.

* **Silhouette Score (0.3492):** This is a fair to weak score. It indicates that while the model has found some structure in the data, the four student profiles have a notable amount of overlap and are not sharply separated from one another.
* **Cluster Distribution:** The model has created four large and relatively balanced clusters. This is a positive outcome, as it means each group is substantial enough for meaningful analysis, with no tiny, insignificant clusters.

The next critical step is to perform a deeper analysis to understand the unique characteristics of each of these four clusters and define the student "profiles" they represent.

### 3.4.3 Analyzing Cluster Profiles

The final and most important step in our clustering analysis is to interpret what each cluster represents. We do this by calculating the average value of our key features for each of the four clusters.

By examining the average **Depression Rate**, **CGPA**, **Academic Pressure**, and **Financial Stress** for each group, we can build a "profile" or "persona" for the students within them. This will allow us to identify which clusters represent at-risk students and understand the factors that characterize them.

In [ ]:
# --- 1. Calculate the Average of Key Features for Each Cluster ---
# We group by the 'prediction' column (our cluster ID)
# Then, we calculate the mean for our most important columns.
cluster_profiles = clustered_df.groupBy("prediction").agg(
    avg(col("label")).alias("Depression_Rate"),
    avg(col("CGPA")).alias("Average_CGPA"),
    avg(col("Academic Pressure")).alias("Avg_Academic_Pressure"),
    avg(col("Financial Stress")).alias("Avg_Financial_Stress")
).orderBy("prediction")

# --- 2. Display the Results in a Clean Table ---
print("--- Student Cluster Profiles ---")
# Convert to a Pandas DataFrame for better formatting and easier analysis
cluster_profiles_pd = cluster_profiles.toPandas()
display(cluster_profiles_pd)

### Interpreting the Student Cluster Profiles 

This table is the key to understanding our four student clusters. By comparing the average values, we can see two distinct types of student profiles emerge: an **"At-Risk"** group and a **"More Resilient"** group.

* **Clusters 1 & 3: The At-Risk Group**
    * **High Depression Rate:** These two groups have a very high prevalence of depression, at **~69%**.
    * **High Stressors:** This high depression rate is strongly correlated with higher average **Academic Pressure (~3.24)** and **Financial Stress (~3.28)**.
    * **Profile:** These clusters represent students who are under significant academic and financial strain, which appears to be a major factor in their mental health.

* **Clusters 0 & 2: The More Resilient Group**
    * **Lower Depression Rate:** These groups have a significantly lower (though still notable) prevalence of depression, ranging from **42% to 49%**.
    * **Lower Stressors:** This corresponds with lower average **Academic Pressure (~3.0)** and **Financial Stress (~2.9)**.
    * **Profile:** These clusters represent students who experience less external pressure, which correlates with better mental health outcomes.

Crucially, the **Average CGPA** is almost identical across all four clusters, suggesting that academic performance itself is not the defining characteristic of these groups. Instead, the key differentiators are the **perceived levels of stress**. This is a powerful insight, as it validates our clustering model and identifies clear, actionable factors that correlate with student depression.

### 3.5 Functionality 4: Association Rule Mining - Discovering Hidden Patterns 🔗

Our fourth and final analytical task is **Association Rule Mining**, another form of unsupervised learning. The goal here is to discover hidden "if-then" rules or relationships within our dataset. For example, we might find a rule like `{Low Sleep, High Financial Stress} => {Depressed}`.

We will use the **FPGrowth** algorithm in Spark MLlib. This algorithm requires the data to be in a specific format: each row must have a single column containing a list (or "basket") of categorical "items". To achieve this, our workflow will be:

1.  **Discretize Numerical Data:** We must convert our continuous numerical features (like `CGPA` and `Academic Pressure`) into discrete categories (like 'Low_CGPA', 'High_Pressure'). We'll use the `QuantileDiscretizer` to automatically create meaningful bins (e.g., Low, Medium, High).
2.  **Combine All Features:** We will then combine these new discretized features with our original categorical features into a single "items" list for each student.
3.  **Train FPGrowth:** We'll run the FPGrowth model on this prepared data to generate the rules.
4.  **Interpret the Rules:** We will analyze the generated rules based on three key metrics:
    * **Antecedent (if):** The item or set of items on the left side of the rule.
    * **Consequent (then):** The item on the right side of the rule.
    * **Confidence:** The probability that the consequent occurs, given the antecedent. A confidence of 0.8 means the rule is correct 80% of the time.
    * **Lift:** How much more likely the consequent is to occur when the antecedent is present. A lift greater than 1 indicates a meaningful relationship.

In [ ]:
# --- 1. Prepare Data for FPGrowth ---
print("Preparing data for FPGrowth...")
cols_to_discretize = ['CGPA', 'Academic Pressure', 'Financial Stress']
discretizer_stages = []
for col_name in cols_to_discretize:
    discretizer = QuantileDiscretizer(numBuckets=3, inputCol=col_name, outputCol=col_name + "_cat", handleInvalid="keep")
    discretizer_stages.append(discretizer)

discretizer_pipeline = Pipeline(stages=discretizer_stages)
discretizer_model = discretizer_pipeline.fit(final_df)
discretized_df = discretizer_model.transform(final_df)

all_item_cols = []
for col_name in categorical_cols:
    expr_col = concat(lit(col_name + "_"), col(col_name))
    all_item_cols.append(expr_col)
for col_name in cols_to_discretize:
    expr_col = concat(lit(col_name + "_"), col(col_name + "_cat"))
    all_item_cols.append(expr_col)

items_df = discretized_df.withColumn("items", array(all_item_cols))
print("Data preparation complete.")

# --- 2. Initialize and Train the FPGrowth Model ---
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.3, minConfidence=0.6)
print("\nTraining FPGrowth model...")
fp_model = fpGrowth.fit(items_df)
print("Model training complete.")

# --- 3. Format and Display the Association Rules ---
print("\n--- Top Association Rules (Ordered by Lift) ---")
association_rules = fp_model.associationRules
rules_pd = association_rules.orderBy(col("lift").desc()).limit(20).toPandas()

# Define a more robust function to make the rules human-readable
def format_rule_corrected(items):
    level_map = {"_0.0": " (Low)", "_1.0": " (Medium)", "_2.0": " (High)"}
    cleaned_items = []
    for item in items:
        # Handle specific long questions first to avoid errors
        if "Have you ever had suicidal thoughts ?_Yes" in item:
            cleaned_items.append("Suicidal Thoughts: Yes")
            continue
        if "Have you ever had suicidal thoughts ?_No" in item:
            cleaned_items.append("Suicidal Thoughts: No")
            continue

        # Handle other specific prefixes
        item = item.replace("Family History of Mental Illness_", "Family History: ")

        # Replace level numbers with names for discretized columns
        for code, name in level_map.items():
            item = item.replace(code, name)

        # General formatting
        item = item.replace("_", " ")
        cleaned_items.append(item)
    return " AND ".join(cleaned_items)

# Apply the corrected formatting function
rules_pd['antecedent'] = rules_pd['antecedent'].apply(format_rule_corrected)
rules_pd['consequent'] = rules_pd['consequent'].apply(format_rule_corrected)

# Rename columns for clarity
rules_pd.rename(columns={'antecedent': 'IF', 'consequent': 'THEN'}, inplace=True)

# Format the numeric columns
rules_pd['confidence'] = rules_pd['confidence'].map('{:.2%}'.format)
rules_pd['lift'] = rules_pd['lift'].map('{:.2f}'.format)
rules_pd['support'] = rules_pd['support'].map('{:.2%}'.format)

display(rules_pd)

### Interpreting the Association Rules 

The FPGrowth algorithm has successfully uncovered several powerful "if-then" rules within our student data. The results have been formatted for clarity, with the "if" condition (`antecedent`) and the "then" outcome (`consequent`) clearly displayed.

To interpret these rules, we focus on two key metrics:
* **Confidence:** The accuracy of the rule. A confidence of 75% means that if the **IF** condition is met, the **THEN** outcome will be true 75% of the time.
* **Lift:** The importance of the rule. A lift greater than 1 shows a real relationship. A lift of 1.19 means the **THEN** outcome is **19% more likely** to happen when the **IF** condition is true.

#### Key Insights Discovered 
The most significant rules (those with the highest lift) reveal a strong link between stress and suicidal thoughts:

* **Rule 1: IF a student has `High Academic Pressure`, THEN they are likely to have had `suicidal thoughts`.**
    * **Confidence: 75.40%**. This rule is highly accurate.
    * **Lift: 1.19**. This is a strong relationship. Students with high academic pressure are **19% more likely** than the average student to have experienced suicidal thoughts. This is a major finding.

* **Rule 2: IF a student has `High Financial Stress`, THEN they are likely to have had `suicidal thoughts`.**
    * **Confidence: 72.87%**. This rule is also very accurate.
    * **Lift: 1.15**. Students with high financial stress are **15% more likely** to have had suicidal thoughts.

#### Trivial Rules (The "Student" Rules)
Many of the other rules have a lift very close to 1.0, which indicates a weak or non-existent relationship. These are considered "trivial" rules that confirm the structure of our data (e.g., that nearly everyone is a `Profession Student`) but do not provide new insights.

**Conclusion:** The association rule mining was highly successful. It has statistically validated the powerful and actionable insight that **high academic and financial stress are strongly associated with suicidal thoughts** among students.

## Part 4: In-Depth Analysis & Key Findings 

Now that we have built and evaluated our four models, we move beyond basic implementation to extract deeper, actionable insights from our findings. This section is designed to address the "Excellence and Innovation" criterion by demonstrating a more profound analysis of the data.

Our first step is to conduct a **Feature Importance Analysis**. The goal is to identify which factors in our dataset are the most powerful predictors of student depression. By understanding the key drivers, we can provide more targeted recommendations for potential interventions.

### 4.1 Feature Importance Analysis

To determine which features are most influential, we will extract the **coefficients** from our trained **Logistic Regression model** (`lr_model`). In a logistic regression model, the absolute magnitude of a coefficient corresponds to its importance in the prediction. A large coefficient (either positive or negative) means that a change in that feature has a strong impact on the likelihood of a student being classified as depressed.

We will visualize the top 15 most important features in a bar chart to clearly see the key factors at a glance.

In [ ]:
# --- 1. Extract Feature Names and Coefficients ---
feature_attributes = transformed_df.schema["features"].metadata["ml_attr"]["attrs"]
all_feature_names = [attr["name"] for attr in feature_attributes["numeric"]] + [attr["name"] for attr in feature_attributes["binary"]]
coefficients = lr_model.coefficients.toArray()

# --- 2. Create DataFrame for Analysis ---
feature_importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': np.abs(coefficients)
})

# --- 3. Sort and Select Top 15 Features ---
top_15_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(15)

# --- 4. Define a Function to Clean Feature Names ---
def clean_feature_name(name):
    # Replaces the technical prefixes with a clean, readable format
    name = name.replace("City_Vec_", "City: ")
    name = name.replace("Profession_Vec_", "Profession: ")
    # Removes extra quotes that can appear
    name = name.replace("'", "")
    return name

# Apply the cleaning function to the 'Feature' column
top_15_features['Feature'] = top_15_features['Feature'].apply(clean_feature_name)

# --- 5. Plot the Cleaned Results ---
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=top_15_features, hue='Feature', palette='viridis', legend=False)
plt.title('Top 15 Most Important Features for Predicting Depression', fontsize=16)
plt.xlabel('Coefficient Magnitude (Importance)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.show()

### Interpreting the Feature Importance Results 🎯

This chart reveals the most influential factors the Logistic Regression model uses to make its predictions. The results are both surprising and insightful.

#### Understanding the X-Axis (Importance)
The x-axis, **"Coefficient Magnitude,"** represents the "voting power" of each feature in the model's decision. A longer bar means the feature has a stronger influence; for example, the model pays far more attention to `City: Nalyan` (with a score over 50) than it does to `Profession: Digital Marketer` (with a score below 30).

#### The Dominance of Location 🗺️
The most striking finding is that **specific cities are by far the most powerful predictors of depression**. Features like `City: Nalyan` and `City: Bhavna` have the highest importance scores. This suggests that there are strong underlying geographical or socio-economic factors associated with these specific cities that are heavily correlated with student mental health in this dataset.

#### The Role of Profession 🧑‍🏫
After location, a few specific professions also show some predictive power. The presence of roles like `Doctor` or `Educational Consultant` in a student dataset likely indicates that the data includes postgraduate or part-time students who are also working professionals, adding another layer of complexity to their student life.

**Conclusion:** This deep-dive analysis has uncovered a critical insight that was not obvious from our initial exploration. The model has learned that **"where you are"** is a more significant predictor of depression than many other factors. This is a perfect example of how machine learning can identify complex patterns in data, providing a new direction for further investigation.

### 4.2 "Deep Dive" Visualisation: Stress vs. Depression

To create a powerful, summary visualization for our key findings, we will directly compare the average stress levels of depressed and non-depressed students. Our previous analyses (feature importance and association rules) strongly suggested that **Academic Pressure** and **Financial Stress** are major factors.

This grouped bar chart will provide clear, visual confirmation of this relationship. By plotting the average stress scores for both groups side-by-side, we can instantly see the magnitude of the difference, creating a compelling "wow" factor for the project's conclusion.

In [ ]:
# --- 1. Aggregate the Data in Spark ---
# Group by the 'label' (Depression status) and calculate the average stress levels for each group.
stress_analysis = final_df.groupBy("label").agg(
    avg(col("Academic Pressure")).alias("Academic Pressure"),
    avg(col("Financial Stress")).alias("Financial Stress")
)

# --- 2. Convert to Pandas and Prepare for Plotting ---
# Convert the aggregated Spark DataFrame to a Pandas DataFrame
stress_pd = stress_analysis.toPandas()

# "Melt" the DataFrame to transform it from a "wide" format to a "long" format,
# which is the required input structure for a Seaborn grouped bar plot.
plot_data = pd.melt(
    stress_pd,
    id_vars=['label'],
    var_name='Stress Type',
    value_name='Average Score'
)

# Map the numerical label to a human-readable status for the plot legend
plot_data['Depression Status'] = plot_data['label'].map({0: 'Not Depressed', 1: 'Depressed'})

# --- 3. Create the Grouped Bar Plot ---
plt.figure(figsize=(10, 7))
sns.barplot(
    data=plot_data,
    x='Stress Type',
    y='Average Score',
    hue='Depression Status',
    palette={'Not Depressed': 'blue', 'Depressed': 'red'} # Use the same colors as before
)
plt.title('Average Stress Levels by Depression Status', fontsize=16)
plt.xlabel('Type of Stressor', fontsize=12)
plt.ylabel('Average Stress Score (0-5 Scale)', fontsize=12)
plt.legend(title='Depression Status')
plt.show()

### Interpreting the "Deep Dive" Visualization 📊

This grouped bar chart provides clear and compelling visual evidence for one of the most significant findings of this project: **there is a strong correlation between higher stress levels and the likelihood of depression among students.**

#### Key Observations:
* **Academic Pressure:** The chart shows a stark difference between the two groups. Students classified as **Depressed** (red bar) report a much higher average academic pressure score (around 3.7) compared to those who are **Not Depressed** (blue bar, around 2.4).
* **Financial Stress:** The same strong pattern holds true for financial stress. The **Depressed** group reports a significantly higher average score (around 3.6) than the **Not Depressed** group (around 2.5).

**Conclusion:** This single visualization effectively encapsulates the insights from both our feature importance analysis and our association rule mining. It powerfully demonstrates that students experiencing higher levels of academic and financial pressure are far more likely to also report being depressed. This serves as an excellent summary and "wow factor" for the project's conclusion.

### Additional Visualization: The Impact of Family History

To supplement our primary finding on stress, we will also visualize the impact of another key variable identified in our earlier analyses: `Family History of Mental Illness`. The following 100% stacked bar chart illustrates the difference in depression rates between students with and without a family history of mental illness, providing another layer of insight into the factors at play.

In [ ]:
# --- 1. Aggregate the Data in Spark ---
# Group by family history and depression status to get counts
family_history_counts = final_df.groupBy("Family History of Mental Illness", "label").count()

# --- 2. Convert to Pandas and Calculate Proportions ---
family_history_pd = family_history_counts.toPandas()

# Calculate the total number of students for each family history category (Yes/No)
totals = family_history_pd.groupby('Family History of Mental Illness')['count'].sum()

# Calculate the percentage for each group
family_history_pd['percentage'] = family_history_pd.apply(lambda row: 100 * row['count'] / totals[row['Family History of Mental Illness']], axis=1)

# --- 3. Create the 100% Stacked Bar Plot ---
# Pivot the data to get it into the right shape for plotting
pivot_df = family_history_pd.pivot(index='Family History of Mental Illness', columns='label', values='percentage')

# Create the plot
pivot_df.plot(
    kind='bar',
    stacked=True,
    color=['blue', 'red'], # Not Depressed (0), Depressed (1)
    figsize=(10, 7)
)

plt.title('Impact of Family History on Depression Rate', fontsize=16)
plt.xlabel('Family History of Mental Illness', fontsize=12)
plt.ylabel('Percentage of Students', fontsize=12)
plt.xticks(rotation=0) # Keep the x-axis labels horizontal
plt.legend(title='Depression Status', labels=['Not Depressed', 'Depressed'])
plt.show()

### Interpreting the Family History Visualization 🧬

This 100% stacked bar chart effectively visualizes the impact of having a family history of mental illness on the likelihood of a student being depressed.

#### Key Observations:
* **For students with NO family history:** The chart shows that the depression rate is approximately **56%** (the red portion of the "No" bar).
* **For students with a family history:** The depression rate increases to approximately **61%** (the red portion of the "Yes" bar).

**Conclusion:** This chart confirms that there is a **positive correlation between having a family history of mental illness and the rate of depression** in this dataset. While the effect is not as dramatic as the differences seen with the stress-level chart, it is still a significant finding that validates one of the key features identified in our earlier analyses. This serves as a great secondary insight for the project.

## Part 5: Conclusion & Future Work

This project successfully leveraged Apache Spark and its MLlib library to conduct a comprehensive analysis of student mental health. By implementing four distinct machine learning functionalities, we were able to extract a wide range of valuable insights from the dataset.

### 5.1 Summary of Findings 

Our analysis yielded several key findings:
* **Strong Predictive Power for Depression:** Our Logistic Regression model proved to be highly effective at predicting student depression, achieving an **AUC of 0.92** and a **Recall of 88.24%**. The feature importance analysis revealed that a student's **city of residence** was the most powerful predictor.
* **Stress as a Key Factor:** Both the **Association Rule Mining** and the **"Deep Dive" Visualisation** confirmed a strong, statistically significant link between high levels of **Academic and Financial Stress** and the likelihood of depression and suicidal thoughts.
* **Identifiable At-Risk Profiles:** The **KMeans Clustering** analysis successfully identified distinct student profiles. The "At-Risk" groups were clearly characterized by higher average stress levels, validating the findings from our other models.
* **Poor Predictability of Academic Performance:** Our **Linear Regression** model demonstrated that the available features were very poor predictors of a student's CGPA, achieving an **R² of only 0.0168**. This is a valuable finding in itself, indicating that academic performance is likely influenced by factors not captured in this dataset.

### 5.2 Project Reflection on Cloud Technologies 

This project demonstrated the immense value of using a distributed computing framework like **Apache Spark**. For a dataset of nearly 28,000 records with numerous features, Spark allowed for:
* **Scalability:** All data loading, pre-processing, and model training tasks were performed efficiently. This architecture could seamlessly scale to handle a dataset with millions of student records without requiring significant code changes.
* **Unified Framework:** Spark provided a single, unified API to handle everything from initial data ingestion (Spark SQL) to complex machine learning (Spark MLlib). This streamlined the entire workflow and is a significant advantage over traditional tools that would require separate libraries for each step.

### 5.3 Future Work 

While this project was successful, there are several exciting avenues for future work:
* **Model Deployment:** The trained Logistic Regression model could be deployed as a real-time web application or API. This could serve as the backend for a student wellness tool that provides early-intervention recommendations.
* **Advanced Algorithms:** More complex models, such as Gradient-Boosted Trees or Random Forests, could be trained to potentially improve predictive accuracy and capture non-linear relationships in the data.
* **Deeper Feature Engineering:** Incorporating additional data sources, such as social media activity or university attendance records, could provide new features that might improve the predictability of both depression and academic performance.